In [1]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

import numpy as np
from sklearn import datasets

from core.io.migrate import migrate
from core.graph.node import Node
from core.graph.node_set import NodeSet, NodeSetType
from core.graph.edge import Edge
from core.graph.versioned_var import VersionedVariable
from core.container import DataContainer

from core.record_event import RecordEvent, ClearEvent
from core.event import operation_events

ClearEvent()

In [2]:
@RecordEvent
def cell_2():
    iris_X, iris_y = datasets.load_iris(return_X_y=True)
    return iris_X, iris_y

In [3]:
iris_X, iris_y = cell_2()

In [4]:
@RecordEvent
def cell_4():
    indices = np.random.permutation(len(iris_X))
    iris_X_train = iris_X[indices[:-10]]
    iris_y_train = iris_y[indices[:-10]]
    iris_X_test = iris_X[indices[-10:]]
    iris_y_test = iris_y[indices[-10:]]
    return indices, iris_X_train, iris_y_train, iris_X_test, iris_y_test

In [5]:
indices, iris_X_train, iris_y_train, iris_X_test, iris_y_test = cell_4()

In [6]:
node1 = Node(VersionedVariable("iris_X", 1, 1), DataContainer("iris_X", 1, iris_X, operation_events[0]), 1)
node2 = Node(VersionedVariable("iris_y", 2, 1), DataContainer("iris_y", 1, iris_y, operation_events[0]), 1)
node3 = Node(VersionedVariable("indices", 3, 1), DataContainer("indices", 1, indices, operation_events[1]), 1)
node4 = Node(VersionedVariable("iris_X_train", 4, 1), DataContainer("iris_X_train", 1, iris_X_train, operation_events[1]), 1)
node5 = Node(VersionedVariable("iris_y_train", 5, 1), DataContainer("iris_y_train", 1, iris_y_train, operation_events[1]), 1)
node6 = Node(VersionedVariable("iris_X_test", 6, 1), DataContainer("iris_X_test", 1, iris_X_test, operation_events[1]), 1)
node7 = Node(VersionedVariable("iris_y_test", 7, 1), DataContainer("iris_y_test", 1, iris_y_test, operation_events[1]), 1)
edge = Edge(operation_events[1], "", NodeSet([node1, node2], NodeSetType.INPUT), NodeSet([node3, node4, node5, node6, node7], NodeSetType.OUTPUT))

In [7]:
migrate([node1, node2], [edge], "test")

['iris_X = data_container_dict["iris_X"]',
 'iris_y = data_container_dict["iris_y"]',
 'def cell_4():\n    indices = np.random.permutation(len(iris_X))\n    iris_X_train = iris_X[indices[:-10]]\n    iris_y_train = iris_y[indices[:-10]]\n    iris_X_test = iris_X[indices[-10:]]\n    iris_y_test = iris_y[indices[-10:]]\n    return indices, iris_X_train, iris_y_train, iris_X_test, iris_y_test',
 'indices, iris_X_train, iris_y_train, iris_X_test, iris_y_test = cell_4()']